In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 500
import matplotlib.pyplot as plt
#matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error as mae

import catboost as cb

import warnings
warnings.simplefilter("ignore")

In [2]:
# путь расположения выборки
path_train = '/home/tefi/Homework/Sport/assignment_2.csv'

# загрузка данных из файла
train = pd.read_csv(path_train)
train

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.000000,0.0,0.0,0.000000,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.

In [3]:
train_target = train.copy()
train_target = train['TransactionAmt']                # целевая переменная для тренировочных данных
train_target.head(3)

0    68.5
1    29.0
2    59.0
Name: TransactionAmt, dtype: float64

In [4]:
train_data = train.copy()
train_data = train_data.drop(['TransactionAmt'], axis = 'columns')

In [5]:
train_data = train_data.select_dtypes(include = ['float64', 'int64'])
train_data

,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,13926,NaN,150.0,142.0,315.0,87.0,19.0,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.000000,0.0,0.0,0.000000,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2755,404.0,150.0,102.0,325.0,87.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [6]:
train_data.isnull().sum()

card1         0
card2      2611
card3         3
card5       953
addr1     19433
          ...  
V335     132004
V336     132004
V337     132004
V338     132004
V339     132004
Length: 376, dtype: int64

In [7]:
f = train_data.columns
f

Index(['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2',
       'C1', 'C2',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=376)

In [8]:
for i in f:
    train_data.loc[(train_data[i].isnull(), [i])] = train_data[i].median()
train_data

,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,13926.0,375.0,150.0,142.0,315.0,87.0,19.0,27.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,100.0,13.0,17.0,9.0,0.0,0.0,51.333332,0.666666,13.0,13.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.000000,0.0,0.0,0.000000,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2755.0,404.0,150.0,102.0,325.0,87.0,9.0,27.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,100.0,8.0,0.0,9.0,0.0,0.0,51.333332,0.666666,0.0,24.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000

In [9]:
# делим выборки на train и test
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.30)

In [10]:
cb_params = {
    "n_estimators": 5000,
    "loss_function": 'MAE',
    "eval_metric": "MAE",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

In [11]:
model = cb.CatBoostRegressor(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train), (x_test, y_test)])

0:	learn: 87.5400133	test: 87.5400133	test1: 88.2884300	best: 88.2884300 (0)	total: 186ms	remaining: 15m 27s
10:	learn: 84.7125954	test: 84.7125954	test1: 85.3825717	best: 85.3825717 (10)	total: 1.33s	remaining: 10m 1s
20:	learn: 82.5181611	test: 82.5181611	test1: 83.1218587	best: 83.1218587 (20)	total: 2.37s	remaining: 9m 21s
30:	learn: 80.8435555	test: 80.8435555	test1: 81.4034072	best: 81.4034072 (30)	total: 3.44s	remaining: 9m 12s
40:	learn: 79.5592120	test: 79.5592120	test1: 80.0945328	best: 80.0945328 (40)	total: 4.55s	remaining: 9m 10s
50:	learn: 78.3696416	test: 78.3696416	test1: 78.8627861	best: 78.8627861 (50)	total: 5.58s	remaining: 9m 1s
60:	learn: 77.3634459	test: 77.3634459	test1: 77.8331846	best: 77.8331846 (60)	total: 6.61s	remaining: 8m 55s
70:	learn: 76.4329766	test: 76.4329766	test1: 76.8882914	best: 76.8882914 (70)	total: 7.63s	remaining: 8m 49s
80:	learn: 75.5703558	test: 75.5703558	test1: 76.0293917	best: 76.0293917 (80)	total: 8.63s	remaining: 8m 44s
90:	learn: 7

740:	learn: 66.3466525	test: 66.3466525	test1: 67.1753570	best: 67.1753570 (740)	total: 1m 20s	remaining: 7m 45s
750:	learn: 66.3128698	test: 66.3128698	test1: 67.1459487	best: 67.1459487 (750)	total: 1m 22s	remaining: 7m 45s
760:	learn: 66.2808509	test: 66.2808509	test1: 67.1186013	best: 67.1186013 (760)	total: 1m 23s	remaining: 7m 43s
770:	learn: 66.2560829	test: 66.2560829	test1: 67.0958178	best: 67.0958178 (770)	total: 1m 24s	remaining: 7m 42s
780:	learn: 66.2264414	test: 66.2264414	test1: 67.0733157	best: 67.0733157 (780)	total: 1m 25s	remaining: 7m 40s
790:	learn: 66.2034747	test: 66.2034747	test1: 67.0532770	best: 67.0532770 (790)	total: 1m 26s	remaining: 7m 39s
800:	learn: 66.1614792	test: 66.1614792	test1: 67.0153732	best: 67.0153732 (800)	total: 1m 27s	remaining: 7m 38s
810:	learn: 66.1348375	test: 66.1348375	test1: 66.9960675	best: 66.9960675 (810)	total: 1m 28s	remaining: 7m 37s
820:	learn: 66.1103921	test: 66.1103921	test1: 66.9752636	best: 66.9752636 (820)	total: 1m 29s	r

1460:	learn: 64.8232587	test: 64.8232587	test1: 65.9499758	best: 65.9499758 (1460)	total: 2m 39s	remaining: 6m 25s
1470:	learn: 64.8019889	test: 64.8019889	test1: 65.9322964	best: 65.9322964 (1470)	total: 2m 40s	remaining: 6m 24s
1480:	learn: 64.7898899	test: 64.7898899	test1: 65.9218938	best: 65.9218938 (1480)	total: 2m 41s	remaining: 6m 23s
1490:	learn: 64.7804194	test: 64.7804194	test1: 65.9169927	best: 65.9169293 (1489)	total: 2m 42s	remaining: 6m 22s
1500:	learn: 64.7748705	test: 64.7748705	test1: 65.9135674	best: 65.9135674 (1500)	total: 2m 43s	remaining: 6m 21s
1510:	learn: 64.7588673	test: 64.7588673	test1: 65.8966049	best: 65.8966049 (1510)	total: 2m 44s	remaining: 6m 20s
1520:	learn: 64.7489548	test: 64.7489548	test1: 65.8897376	best: 65.8897376 (1520)	total: 2m 45s	remaining: 6m 18s
1530:	learn: 64.7334373	test: 64.7334373	test1: 65.8783679	best: 65.8783679 (1530)	total: 2m 46s	remaining: 6m 17s
1540:	learn: 64.7204237	test: 64.7204237	test1: 65.8702181	best: 65.8702181 (154

2180:	learn: 64.0858307	test: 64.0858307	test1: 65.4464335	best: 65.4464335 (2180)	total: 3m 55s	remaining: 5m 4s
2190:	learn: 64.0803054	test: 64.0803054	test1: 65.4433937	best: 65.4433937 (2190)	total: 3m 56s	remaining: 5m 3s
2200:	learn: 64.0737250	test: 64.0737250	test1: 65.4398975	best: 65.4398975 (2200)	total: 3m 57s	remaining: 5m 2s
2210:	learn: 64.0695400	test: 64.0695400	test1: 65.4385638	best: 65.4385001 (2208)	total: 3m 58s	remaining: 5m 1s
2220:	learn: 64.0542283	test: 64.0542283	test1: 65.4252681	best: 65.4252681 (2220)	total: 4m	remaining: 5m
2230:	learn: 64.0486791	test: 64.0486791	test1: 65.4223784	best: 65.4223784 (2230)	total: 4m 1s	remaining: 4m 59s
2240:	learn: 64.0421234	test: 64.0421234	test1: 65.4184889	best: 65.4184889 (2240)	total: 4m 2s	remaining: 4m 58s
2250:	learn: 64.0329571	test: 64.0329571	test1: 65.4138571	best: 65.4138571 (2250)	total: 4m 3s	remaining: 4m 57s
2260:	learn: 64.0266149	test: 64.0266149	test1: 65.4105638	best: 65.4105638 (2260)	total: 4m 4s

2900:	learn: 63.6114123	test: 63.6114123	test1: 65.1794929	best: 65.1794929 (2900)	total: 5m 10s	remaining: 3m 44s
2910:	learn: 63.6045777	test: 63.6045777	test1: 65.1771287	best: 65.1770718 (2909)	total: 5m 11s	remaining: 3m 43s
2920:	learn: 63.6010234	test: 63.6010234	test1: 65.1749209	best: 65.1749209 (2920)	total: 5m 12s	remaining: 3m 42s
2930:	learn: 63.5973078	test: 63.5973078	test1: 65.1734401	best: 65.1732283 (2928)	total: 5m 13s	remaining: 3m 41s
2940:	learn: 63.5888328	test: 63.5888328	test1: 65.1669672	best: 65.1669672 (2940)	total: 5m 14s	remaining: 3m 40s
2950:	learn: 63.5858478	test: 63.5858478	test1: 65.1654828	best: 65.1654528 (2947)	total: 5m 15s	remaining: 3m 39s
2960:	learn: 63.5814605	test: 63.5814605	test1: 65.1635002	best: 65.1634063 (2959)	total: 5m 16s	remaining: 3m 38s
2970:	learn: 63.5777794	test: 63.5777794	test1: 65.1632251	best: 65.1632128 (2961)	total: 5m 17s	remaining: 3m 37s
2980:	learn: 63.5751469	test: 63.5751469	test1: 65.1626065	best: 65.1626065 (298

3620:	learn: 63.3007232	test: 63.3007232	test1: 65.0251904	best: 65.0251904 (3620)	total: 6m 25s	remaining: 2m 26s
3630:	learn: 63.2947361	test: 63.2947361	test1: 65.0207189	best: 65.0207189 (3630)	total: 6m 26s	remaining: 2m 25s
3640:	learn: 63.2873795	test: 63.2873795	test1: 65.0159956	best: 65.0159956 (3640)	total: 6m 27s	remaining: 2m 24s
3650:	learn: 63.2804044	test: 63.2804044	test1: 65.0127635	best: 65.0127635 (3650)	total: 6m 28s	remaining: 2m 23s
3660:	learn: 63.2767667	test: 63.2767667	test1: 65.0121204	best: 65.0120422 (3653)	total: 6m 29s	remaining: 2m 22s
3670:	learn: 63.2720991	test: 63.2720991	test1: 65.0097524	best: 65.0097524 (3670)	total: 6m 30s	remaining: 2m 21s
3680:	learn: 63.2675756	test: 63.2675756	test1: 65.0064467	best: 65.0064467 (3680)	total: 6m 31s	remaining: 2m 20s
3690:	learn: 63.2635807	test: 63.2635807	test1: 65.0049897	best: 65.0049897 (3690)	total: 6m 32s	remaining: 2m 19s
3700:	learn: 63.2579656	test: 63.2579656	test1: 65.0029272	best: 65.0029272 (370

4340:	learn: 63.0249546	test: 63.0249546	test1: 64.9056724	best: 64.9056724 (4340)	total: 7m 39s	remaining: 1m 9s
4350:	learn: 63.0228551	test: 63.0228551	test1: 64.9048981	best: 64.9048981 (4350)	total: 7m 40s	remaining: 1m 8s
4360:	learn: 63.0214718	test: 63.0214718	test1: 64.9045258	best: 64.9045258 (4360)	total: 7m 41s	remaining: 1m 7s
4370:	learn: 63.0185173	test: 63.0185173	test1: 64.9040188	best: 64.9040102 (4365)	total: 7m 42s	remaining: 1m 6s
4380:	learn: 63.0134350	test: 63.0134350	test1: 64.9017278	best: 64.9017278 (4380)	total: 7m 43s	remaining: 1m 5s
4390:	learn: 63.0026785	test: 63.0026785	test1: 64.8927270	best: 64.8925870 (4389)	total: 7m 44s	remaining: 1m 4s
4400:	learn: 63.0011286	test: 63.0011286	test1: 64.8920101	best: 64.8919471 (4398)	total: 7m 45s	remaining: 1m 3s
4410:	learn: 62.9997813	test: 62.9997813	test1: 64.8913260	best: 64.8913260 (4410)	total: 7m 46s	remaining: 1m 2s
4420:	learn: 62.9974120	test: 62.9974120	test1: 64.8904987	best: 64.8904987 (4420)	total

In [12]:
train_data_c = train.copy()
train_data_c = train_data_c.select_dtypes(include = ['object'])
train_data_c

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN
2,W,visa,debit,outlook.com,NaN,T,T,T,M0,F,F,F,F,F
3,W,mastercard,debit,yahoo.com,NaN,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
4,H,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,W,mastercard,debit,gmail.com,NaN,T,T,F,M1,T,F,NaN,NaN,NaN
179996,W,mastercard,debit,yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN
179997,W,mastercard,debit,gmail.com,NaN,T,T,T,M0,T,F,NaN,NaN,NaN
179998,W,visa,debit,NaN,NaN,NaN,NaN,NaN,M1,T,T,NaN,NaN,NaN


In [13]:
cat = train_data_c.columns
for i in cat:
    train_data_c.loc[(train_data_c[i].isnull(), [i])] = train_data_c[i].mode()[0]
train_data_c

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,gmail.com,gmail.com,T,T,T,M2,F,T,F,F,T
1,W,mastercard,credit,gmail.com,gmail.com,T,T,T,M0,T,T,F,F,T
2,W,visa,debit,outlook.com,gmail.com,T,T,T,M0,F,F,F,F,F
3,W,mastercard,debit,yahoo.com,gmail.com,T,T,T,M0,T,F,F,F,T
4,H,mastercard,credit,gmail.com,gmail.com,T,T,T,M0,F,F,F,F,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,W,mastercard,debit,gmail.com,gmail.com,T,T,F,M1,T,F,F,F,T
179996,W,mastercard,debit,yahoo.com,gmail.com,T,T,T,M0,F,T,F,F,T
179997,W,mastercard,debit,gmail.com,gmail.com,T,T,T,M0,T,F,F,F,T
179998,W,visa,debit,gmail.com,gmail.com,T,T,T,M1,T,T,F,F,T


In [14]:
for i in cat:
    train_data_c = pd.concat([train_data_c, pd.get_dummies(train_data_c[i], prefix=i)], axis=1)
    train_data_c = train_data_c.drop(i, axis = 'columns')
train_data_c

,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_charge card,card6_credit,card6_debit,card6_debit or credit,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmail.com,R_emaildomain_hotmail.de,R_emaildomain_hotmail.es,R_emaildomain_hotmail.fr,R_emaildomain_icloud.com,R_emaildomain_juno.com,R_emaildomain_live.com,R_emaildomain_live.com.mx,R_emaildomain_live.fr,R_emaildomain_mac.com,R_emaildomain_mail.com,R_emaildomain_me.com,R_emaildomain_msn.com,R_emaildomain_netzero.com,R_emaildomain_netzero.net,R_emaildomain_optonline.net,R_emaildomain_outlook.com,R_emaildomain_outlook.es,R_emaildomain_prodigy.net.mx,R_emaildomain_protonmail.com,R_emaildomain_ptd.net,R_emaildomain_q.com,R_emaildomain_roadrunner.com,R_emaildomain_rocketmail.com,R_emaildomain_sbcglobal.net,R_emaildomain_sc.rr.com,R_emaildomain_scranton.edu,R_emaildomain_servicios-ta.com,R_emaildomain_suddenlink.net,R_emaildomain_twc.com,R_emaildomain_verizon.net,R_emaildomain_web.de,R_emaildomain_windstream.net,R_emaildomain_yahoo.co.jp,R_emaildomain_yahoo.co.uk,R_emaildomain_yahoo.com,R_emaildomain_yahoo.com.mx,R_emaildomain_yahoo.de,R_emaildomain_yahoo.es,R_emaildomain_yahoo.fr,R_emaildomain_ymail.com,M1_F,M1_T,M2_F,M2_T,M3_F,M3_T,M4_M0,M4_M1,M4_M2,M5_F,M5_T,M6_F,M6_T,M7_F,M7_T,M8_F,M8_T,M9_F,M9_T
0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1
1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,1,0,1,1,0,1,0,0,1
2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [15]:
train_data = pd.concat([train_data, train_data_c], axis=1)
train_data

,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,...,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_charge card,card6_credit,card6_debit,card6_debit or credit,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmail.com,R_emaildomain_hotmail.de,R_emaildomain_hotmail.es,R_emaildomain_hotmail.fr,R_emaildomain_icloud.com,R_emaildomain_juno.com,R_emaildomain_live.com,R_emaildomain_live.com.mx,R_emaildomain_live.fr,R_emaildomain

In [16]:
# делим выборки на train и test
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.30)

In [17]:
cb_params = {
    "n_estimators": 5000,
    "loss_function": 'MAE',
    "eval_metric": "MAE",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

In [18]:
model = cb.CatBoostRegressor(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train), (x_test, y_test)])

0:	learn: 88.2644276	test: 88.2644276	test1: 86.5512690	best: 86.5512690 (0)	total: 112ms	remaining: 9m 20s
10:	learn: 84.1659280	test: 84.1659280	test1: 82.5233413	best: 82.5233413 (10)	total: 1.33s	remaining: 10m 3s
20:	learn: 81.3843490	test: 81.3843490	test1: 79.8273989	best: 79.8273989 (20)	total: 2.53s	remaining: 10m
30:	learn: 79.3269047	test: 79.3269047	test1: 77.8485814	best: 77.8485814 (30)	total: 3.78s	remaining: 10m 6s
40:	learn: 77.8289736	test: 77.8289736	test1: 76.4251939	best: 76.4251939 (40)	total: 4.92s	remaining: 9m 55s
50:	learn: 76.4314134	test: 76.4314134	test1: 75.0998506	best: 75.0998506 (50)	total: 6.12s	remaining: 9m 53s
60:	learn: 75.2743180	test: 75.2743180	test1: 74.0089744	best: 74.0089744 (60)	total: 7.24s	remaining: 9m 45s
70:	learn: 74.3665410	test: 74.3665410	test1: 73.1522843	best: 73.1522843 (70)	total: 8.36s	remaining: 9m 40s
80:	learn: 73.6580617	test: 73.6580617	test1: 72.4758345	best: 72.4758345 (80)	total: 9.62s	remaining: 9m 44s
90:	learn: 72.9

740:	learn: 63.5227021	test: 63.5227021	test1: 63.2173142	best: 63.2173142 (740)	total: 1m 25s	remaining: 8m 9s
750:	learn: 63.5054464	test: 63.5054464	test1: 63.2022960	best: 63.2022960 (750)	total: 1m 26s	remaining: 8m 7s
760:	learn: 63.4746295	test: 63.4746295	test1: 63.1750895	best: 63.1750895 (760)	total: 1m 27s	remaining: 8m 6s
770:	learn: 63.4533794	test: 63.4533794	test1: 63.1570681	best: 63.1570681 (770)	total: 1m 28s	remaining: 8m 4s
780:	learn: 63.4083057	test: 63.4083057	test1: 63.1179284	best: 63.1179284 (780)	total: 1m 29s	remaining: 8m 3s
790:	learn: 63.3835430	test: 63.3835430	test1: 63.0988395	best: 63.0988395 (790)	total: 1m 30s	remaining: 8m 1s
800:	learn: 63.3559605	test: 63.3559605	test1: 63.0784902	best: 63.0784902 (800)	total: 1m 31s	remaining: 8m
810:	learn: 63.3237268	test: 63.3237268	test1: 63.0550846	best: 63.0550846 (810)	total: 1m 32s	remaining: 7m 59s
820:	learn: 63.3060769	test: 63.3060769	test1: 63.0410300	best: 63.0410300 (820)	total: 1m 33s	remaining: 

1460:	learn: 62.0980245	test: 62.0980245	test1: 62.1520437	best: 62.1520437 (1460)	total: 2m 44s	remaining: 6m 39s
1470:	learn: 62.0799229	test: 62.0799229	test1: 62.1375174	best: 62.1375174 (1470)	total: 2m 45s	remaining: 6m 37s
1480:	learn: 62.0637745	test: 62.0637745	test1: 62.1264042	best: 62.1264042 (1480)	total: 2m 47s	remaining: 6m 36s
1490:	learn: 62.0540816	test: 62.0540816	test1: 62.1199118	best: 62.1199118 (1490)	total: 2m 48s	remaining: 6m 35s
1500:	learn: 62.0485289	test: 62.0485289	test1: 62.1163401	best: 62.1163401 (1500)	total: 2m 49s	remaining: 6m 34s
1510:	learn: 62.0406224	test: 62.0406224	test1: 62.1106081	best: 62.1106081 (1510)	total: 2m 50s	remaining: 6m 32s
1520:	learn: 62.0314421	test: 62.0314421	test1: 62.1039369	best: 62.1039369 (1520)	total: 2m 51s	remaining: 6m 31s
1530:	learn: 62.0223832	test: 62.0223832	test1: 62.0970906	best: 62.0970906 (1530)	total: 2m 52s	remaining: 6m 30s
1540:	learn: 62.0080720	test: 62.0080720	test1: 62.0874499	best: 62.0874499 (154

2180:	learn: 61.4248294	test: 61.4248294	test1: 61.7574155	best: 61.7574155 (2180)	total: 4m 3s	remaining: 5m 14s
2190:	learn: 61.4182871	test: 61.4182871	test1: 61.7545291	best: 61.7545291 (2190)	total: 4m 4s	remaining: 5m 13s
2200:	learn: 61.4099985	test: 61.4099985	test1: 61.7488820	best: 61.7488820 (2200)	total: 4m 5s	remaining: 5m 12s
2210:	learn: 61.4036153	test: 61.4036153	test1: 61.7461324	best: 61.7461324 (2210)	total: 4m 6s	remaining: 5m 11s
2220:	learn: 61.3938316	test: 61.3938316	test1: 61.7425306	best: 61.7425306 (2220)	total: 4m 7s	remaining: 5m 10s
2230:	learn: 61.3857865	test: 61.3857865	test1: 61.7392221	best: 61.7392221 (2230)	total: 4m 8s	remaining: 5m 8s
2240:	learn: 61.3778652	test: 61.3778652	test1: 61.7364985	best: 61.7364985 (2240)	total: 4m 9s	remaining: 5m 7s
2250:	learn: 61.3693244	test: 61.3693244	test1: 61.7308989	best: 61.7308989 (2250)	total: 4m 11s	remaining: 5m 6s
2260:	learn: 61.3643652	test: 61.3643652	test1: 61.7281300	best: 61.7281300 (2260)	total: 

2900:	learn: 60.9703433	test: 60.9703433	test1: 61.5281163	best: 61.5280335 (2895)	total: 5m 21s	remaining: 3m 52s
2910:	learn: 60.9652714	test: 60.9652714	test1: 61.5253405	best: 61.5253405 (2910)	total: 5m 22s	remaining: 3m 51s
2920:	learn: 60.9567358	test: 60.9567358	test1: 61.5223651	best: 61.5223651 (2920)	total: 5m 23s	remaining: 3m 50s
2930:	learn: 60.9527206	test: 60.9527206	test1: 61.5202962	best: 61.5202962 (2930)	total: 5m 24s	remaining: 3m 49s
2940:	learn: 60.9490363	test: 60.9490363	test1: 61.5176301	best: 61.5176301 (2940)	total: 5m 25s	remaining: 3m 48s
2950:	learn: 60.9413054	test: 60.9413054	test1: 61.5130670	best: 61.5130670 (2950)	total: 5m 26s	remaining: 3m 46s
2960:	learn: 60.9329872	test: 60.9329872	test1: 61.5095319	best: 61.5095319 (2960)	total: 5m 27s	remaining: 3m 45s
2970:	learn: 60.9261953	test: 60.9261953	test1: 61.5067285	best: 61.5067285 (2970)	total: 5m 29s	remaining: 3m 44s
2980:	learn: 60.9229692	test: 60.9229692	test1: 61.5053688	best: 61.5053688 (298

3620:	learn: 60.6360111	test: 60.6360111	test1: 61.3829227	best: 61.3829227 (3620)	total: 6m 39s	remaining: 2m 32s
3630:	learn: 60.6244207	test: 60.6244207	test1: 61.3772658	best: 61.3772658 (3630)	total: 6m 40s	remaining: 2m 30s
3640:	learn: 60.6197367	test: 60.6197367	test1: 61.3767757	best: 61.3767023 (3638)	total: 6m 41s	remaining: 2m 29s
3650:	learn: 60.6149588	test: 60.6149588	test1: 61.3747043	best: 61.3747043 (3650)	total: 6m 42s	remaining: 2m 28s
3660:	learn: 60.6112783	test: 60.6112783	test1: 61.3743150	best: 61.3742408 (3659)	total: 6m 43s	remaining: 2m 27s
3670:	learn: 60.6092930	test: 60.6092930	test1: 61.3733495	best: 61.3733495 (3670)	total: 6m 44s	remaining: 2m 26s
3680:	learn: 60.6079703	test: 60.6079703	test1: 61.3729455	best: 61.3729177 (3678)	total: 6m 45s	remaining: 2m 25s
3690:	learn: 60.6046363	test: 60.6046363	test1: 61.3717721	best: 61.3717721 (3690)	total: 6m 46s	remaining: 2m 24s
3700:	learn: 60.6004497	test: 60.6004497	test1: 61.3701054	best: 61.3701054 (370

4340:	learn: 60.3793117	test: 60.3793117	test1: 61.2878374	best: 61.2878374 (4340)	total: 7m 56s	remaining: 1m 12s
4350:	learn: 60.3748720	test: 60.3748720	test1: 61.2864487	best: 61.2864487 (4350)	total: 7m 57s	remaining: 1m 11s
4360:	learn: 60.3715580	test: 60.3715580	test1: 61.2844720	best: 61.2844720 (4360)	total: 7m 58s	remaining: 1m 10s
4370:	learn: 60.3684872	test: 60.3684872	test1: 61.2831110	best: 61.2830901 (4369)	total: 7m 59s	remaining: 1m 9s
4380:	learn: 60.3658257	test: 60.3658257	test1: 61.2822825	best: 61.2822825 (4380)	total: 8m	remaining: 1m 7s
4390:	learn: 60.3630728	test: 60.3630728	test1: 61.2817674	best: 61.2817674 (4390)	total: 8m 1s	remaining: 1m 6s
4400:	learn: 60.3593197	test: 60.3593197	test1: 61.2810860	best: 61.2810810 (4399)	total: 8m 2s	remaining: 1m 5s
4410:	learn: 60.3555474	test: 60.3555474	test1: 61.2790166	best: 61.2790166 (4410)	total: 8m 3s	remaining: 1m 4s
4420:	learn: 60.3499511	test: 60.3499511	test1: 61.2752219	best: 61.2752209 (4418)	total: 8m

In [19]:
train_data_c = train.copy()
train_data_c = train_data_c.select_dtypes(include = ['object'])
train_data_c

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN
2,W,visa,debit,outlook.com,NaN,T,T,T,M0,F,F,F,F,F
3,W,mastercard,debit,yahoo.com,NaN,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
4,H,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,W,mastercard,debit,gmail.com,NaN,T,T,F,M1,T,F,NaN,NaN,NaN
179996,W,mastercard,debit,yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN
179997,W,mastercard,debit,gmail.com,NaN,T,T,T,M0,T,F,NaN,NaN,NaN
179998,W,visa,debit,NaN,NaN,NaN,NaN,NaN,M1,T,T,NaN,NaN,NaN


In [20]:
cat = train_data_c.columns
for i in cat:
    train_data_c.loc[(train_data_c[i].isnull(), [i])] = train_data_c[i].mode()[0]
train_data_c

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,gmail.com,gmail.com,T,T,T,M2,F,T,F,F,T
1,W,mastercard,credit,gmail.com,gmail.com,T,T,T,M0,T,T,F,F,T
2,W,visa,debit,outlook.com,gmail.com,T,T,T,M0,F,F,F,F,F
3,W,mastercard,debit,yahoo.com,gmail.com,T,T,T,M0,T,F,F,F,T
4,H,mastercard,credit,gmail.com,gmail.com,T,T,T,M0,F,F,F,F,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,W,mastercard,debit,gmail.com,gmail.com,T,T,F,M1,T,F,F,F,T
179996,W,mastercard,debit,yahoo.com,gmail.com,T,T,T,M0,F,T,F,F,T
179997,W,mastercard,debit,gmail.com,gmail.com,T,T,T,M0,T,F,F,F,T
179998,W,visa,debit,gmail.com,gmail.com,T,T,T,M1,T,T,F,F,T


In [21]:
train_data = pd.concat([train_data, train_data_c], axis=1)
train_data

,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,...,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_charge card,card6_credit,card6_debit,card6_debit or credit,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmail.com,R_emaildomain_hotmail.de,R_emaildomain_hotmail.es,R_emaildomain_hotmail.fr,R_emaildomain_icloud.com,R_emaildomain_juno.com,R_emaildomain_live.com,R_emaildomain_live.com.mx,R_emaildomain_live.fr,R_emaildomain_mac.com,R_emaildomain_mail.com,R_emaildomain_me.com,R_emaildomain_msn

In [22]:
# делим выборки на train и test
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.30)

In [23]:
cb_params = {
    "n_estimators": 5000,
    "loss_function": 'MAE',
    "eval_metric": "MAE",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostRegressor(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train), (x_test, y_test)], cat_features=cat)

0:	learn: 88.0287262	test: 88.0287801	test1: 86.7535912	best: 86.7535912 (0)	total: 333ms	remaining: 27m 46s
10:	learn: 83.7986319	test: 83.7977744	test1: 82.5946773	best: 82.5946773 (10)	total: 3.4s	remaining: 25m 40s
20:	learn: 81.1172299	test: 81.1150597	test1: 79.9609991	best: 79.9609991 (20)	total: 5.94s	remaining: 23m 28s
30:	learn: 79.3197362	test: 79.3173246	test1: 78.1835708	best: 78.1835708 (30)	total: 8.73s	remaining: 23m 19s
40:	learn: 77.8196382	test: 77.8172234	test1: 76.6931210	best: 76.6931210 (40)	total: 11.4s	remaining: 22m 53s
50:	learn: 76.5690286	test: 76.5658351	test1: 75.4771234	best: 75.4771234 (50)	total: 14s	remaining: 22m 41s
60:	learn: 75.4968784	test: 75.4935519	test1: 74.4228714	best: 74.4228714 (60)	total: 16.7s	remaining: 22m 32s
70:	learn: 74.2089520	test: 74.2054322	test1: 73.1300566	best: 73.1300566 (70)	total: 19.4s	remaining: 22m 23s
80:	learn: 73.3272599	test: 73.3237532	test1: 72.2354104	best: 72.2354104 (80)	total: 22s	remaining: 22m 18s
90:	lear

730:	learn: 63.4195698	test: 63.4063821	test1: 62.8748234	best: 62.8748234 (730)	total: 3m 22s	remaining: 19m 44s
740:	learn: 63.4008825	test: 63.3876873	test1: 62.8592066	best: 62.8592066 (740)	total: 3m 25s	remaining: 19m 43s
750:	learn: 63.3538672	test: 63.3404829	test1: 62.8242437	best: 62.8242437 (750)	total: 3m 28s	remaining: 19m 40s
760:	learn: 63.3222594	test: 63.3088809	test1: 62.8025962	best: 62.8025962 (760)	total: 3m 31s	remaining: 19m 38s
770:	learn: 63.3034664	test: 63.2901328	test1: 62.7872903	best: 62.7872903 (770)	total: 3m 34s	remaining: 19m 35s
780:	learn: 63.2815506	test: 63.2684101	test1: 62.7692089	best: 62.7692089 (780)	total: 3m 36s	remaining: 19m 31s
790:	learn: 63.2625653	test: 63.2493841	test1: 62.7555072	best: 62.7555072 (790)	total: 3m 39s	remaining: 19m 28s
800:	learn: 63.2446433	test: 63.2314765	test1: 62.7400731	best: 62.7400731 (800)	total: 3m 42s	remaining: 19m 25s
810:	learn: 63.2249201	test: 63.2117134	test1: 62.7249537	best: 62.7249537 (810)	total: 

1450:	learn: 62.0612370	test: 62.0481594	test1: 61.8812484	best: 61.8812484 (1450)	total: 6m 40s	remaining: 16m 19s
1460:	learn: 62.0474407	test: 62.0344713	test1: 61.8722591	best: 61.8722591 (1460)	total: 6m 43s	remaining: 16m 16s
1470:	learn: 62.0380728	test: 62.0252035	test1: 61.8677168	best: 61.8677168 (1470)	total: 6m 45s	remaining: 16m 13s
1480:	learn: 62.0233043	test: 62.0106019	test1: 61.8564095	best: 61.8564095 (1480)	total: 6m 48s	remaining: 16m 11s
1490:	learn: 62.0126659	test: 61.9999371	test1: 61.8496983	best: 61.8496983 (1490)	total: 6m 51s	remaining: 16m 8s
1500:	learn: 62.0062637	test: 61.9935480	test1: 61.8448348	best: 61.8448348 (1500)	total: 6m 54s	remaining: 16m 7s
1510:	learn: 61.9911924	test: 61.9786761	test1: 61.8364852	best: 61.8364852 (1510)	total: 6m 57s	remaining: 16m 4s
1520:	learn: 61.9813573	test: 61.9688588	test1: 61.8288671	best: 61.8288671 (1520)	total: 7m	remaining: 16m 2s
1530:	learn: 61.9747895	test: 61.9621804	test1: 61.8256000	best: 61.8256000 (153

2160:	learn: 61.4659945	test: 61.4559365	test1: 61.5294270	best: 61.5294270 (2160)	total: 9m 47s	remaining: 12m 52s
2170:	learn: 61.4603883	test: 61.4504990	test1: 61.5267092	best: 61.5267092 (2170)	total: 9m 50s	remaining: 12m 49s
2180:	learn: 61.4534287	test: 61.4435528	test1: 61.5236311	best: 61.5236311 (2180)	total: 9m 53s	remaining: 12m 46s
2190:	learn: 61.4476507	test: 61.4378004	test1: 61.5217834	best: 61.5217834 (2190)	total: 9m 56s	remaining: 12m 44s
2200:	learn: 61.4432002	test: 61.4334496	test1: 61.5193864	best: 61.5193864 (2200)	total: 9m 58s	remaining: 12m 41s
2210:	learn: 61.4359145	test: 61.4260313	test1: 61.5155365	best: 61.5155365 (2210)	total: 10m 1s	remaining: 12m 38s
2220:	learn: 61.4276445	test: 61.4177260	test1: 61.5117211	best: 61.5117211 (2220)	total: 10m 3s	remaining: 12m 35s
2230:	learn: 61.4209647	test: 61.4114075	test1: 61.5089119	best: 61.5089119 (2230)	total: 10m 6s	remaining: 12m 32s
2240:	learn: 61.4149519	test: 61.4053887	test1: 61.5065340	best: 61.5065

2870:	learn: 61.0855204	test: 61.0809111	test1: 61.3528815	best: 61.3528815 (2870)	total: 12m 51s	remaining: 9m 32s
2880:	learn: 61.0816635	test: 61.0774341	test1: 61.3511405	best: 61.3511405 (2880)	total: 12m 54s	remaining: 9m 29s
2890:	learn: 61.0778690	test: 61.0742406	test1: 61.3498607	best: 61.3496222 (2889)	total: 12m 57s	remaining: 9m 27s
2900:	learn: 61.0737418	test: 61.0701316	test1: 61.3482927	best: 61.3482927 (2900)	total: 13m	remaining: 9m 24s
2910:	learn: 61.0699098	test: 61.0663533	test1: 61.3460809	best: 61.3460809 (2910)	total: 13m 2s	remaining: 9m 21s
2920:	learn: 61.0639613	test: 61.0605357	test1: 61.3424586	best: 61.3424586 (2920)	total: 13m 5s	remaining: 9m 19s
2930:	learn: 61.0587488	test: 61.0552603	test1: 61.3392741	best: 61.3392286 (2929)	total: 13m 8s	remaining: 9m 16s
2940:	learn: 61.0521699	test: 61.0487119	test1: 61.3367813	best: 61.3366775 (2939)	total: 13m 10s	remaining: 9m 13s
2950:	learn: 61.0469770	test: 61.0435691	test1: 61.3345699	best: 61.3345699 (29

3580:	learn: 60.7590357	test: 60.7591345	test1: 61.2190929	best: 61.2190929 (3580)	total: 15m 55s	remaining: 6m 18s
3590:	learn: 60.7567123	test: 60.7568249	test1: 61.2182591	best: 61.2182591 (3590)	total: 15m 57s	remaining: 6m 15s
3600:	learn: 60.7545526	test: 60.7547370	test1: 61.2177160	best: 61.2176975 (3597)	total: 16m	remaining: 6m 13s
3610:	learn: 60.7525786	test: 60.7528395	test1: 61.2169840	best: 61.2169840 (3610)	total: 16m 2s	remaining: 6m 10s
3620:	learn: 60.7470017	test: 60.7473251	test1: 61.2154289	best: 61.2154289 (3620)	total: 16m 5s	remaining: 6m 7s
3630:	learn: 60.7456422	test: 60.7459929	test1: 61.2151045	best: 61.2151045 (3630)	total: 16m 7s	remaining: 6m 4s
3640:	learn: 60.7419055	test: 60.7422459	test1: 61.2127061	best: 61.2127061 (3640)	total: 16m 10s	remaining: 6m 2s
3650:	learn: 60.7391162	test: 60.7395401	test1: 61.2119685	best: 61.2119685 (3650)	total: 16m 12s	remaining: 5m 59s
3660:	learn: 60.7296727	test: 60.7299225	test1: 61.2053575	best: 61.2053575 (3660)

4290:	learn: 60.5001974	test: 60.5049622	test1: 61.1295720	best: 61.1295720 (4290)	total: 18m 55s	remaining: 3m 7s
4300:	learn: 60.4961105	test: 60.5009107	test1: 61.1278508	best: 61.1278491 (4299)	total: 18m 57s	remaining: 3m 4s
4310:	learn: 60.4929934	test: 60.4977577	test1: 61.1265953	best: 61.1265953 (4310)	total: 19m	remaining: 3m 2s
4320:	learn: 60.4888413	test: 60.4937411	test1: 61.1249004	best: 61.1249004 (4320)	total: 19m 2s	remaining: 2m 59s
4330:	learn: 60.4850515	test: 60.4899506	test1: 61.1229242	best: 61.1229242 (4330)	total: 19m 5s	remaining: 2m 56s
4340:	learn: 60.4817521	test: 60.4866978	test1: 61.1210317	best: 61.1210317 (4340)	total: 19m 7s	remaining: 2m 54s
4350:	learn: 60.4788519	test: 60.4837577	test1: 61.1196454	best: 61.1195805 (4349)	total: 19m 10s	remaining: 2m 51s
4360:	learn: 60.4763157	test: 60.4812801	test1: 61.1189803	best: 61.1189793 (4359)	total: 19m 12s	remaining: 2m 48s
4370:	learn: 60.4721291	test: 60.4771207	test1: 61.1172883	best: 61.1172883 (4370)